# Estimate surface volume under a shape layer
<font color='blue'>The following code calculates volume of a DEM under a specified elevation. The DEM is extracted from a larger DEM by clipping with a shapefile. Specific python libraries used to carryout the analysis are *<font color='red'>numpy</font> (for numerical calculation)*, *<font color='red'>matplotlib</font> (for visualization)*, *<font color='red'>rioxarray</font> (for raster analysis)*, *<font color='red'>geopandas</font> (for handling of shapefiles)* and *<font color='red'>pandas</font> (to save a result if required)*.</font>

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import rioxarray as rxr

In [2]:
DEM_in = "https://raw.githubusercontent.com/ramendra1990/gh-estimate_volume/main/dem01_projected.tif"
#DEM_in = "dem01_projected_subset.tif"
dem = rxr.open_rasterio(DEM_in)

In [3]:
dem

<xarray.DataArray (band: 1, y: 3305, x: 3584)> Size: 24MB
[11845120 values with dtype=int16]
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 29kB 3.135e+05 3.135e+05 ... 4.209e+05 4.21e+05
  * y            (y) float64 26kB 2.223e+06 2.223e+06 ... 2.124e+06 2.124e+06
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -32768
    scale_factor:   1.0
    add_offset:     0.0

In [4]:
feat_in = "https://raw.githubusercontent.com/ramendra1990/gh-estimate_volume/main/new_layers_combined.shp"
gdf = gpd.read_file(feat_in)

In [5]:
gdf

,id,layer,path,geometry
0,1,new_layer01,D:/ramendra_EPM103/NGRI/Ajit/new_layer01.shp,"POLYGON ((351541.193 2172263.866, 351733.225 2..."
1,1,new_layer02,D:/ramendra_EPM103/NGRI/Ajit/new_layer02.shp,"POLYGON ((348492.125 2186758.589, 348309.866 2..."


In [ ]:
# Main line of codes
# We need a crs (coordinate refrence system) to be assigned to each feature layers
crs_sub = "epsg:" + str(gdf.crs.to_epsg())
# To save the output, i.e. 3D surface volume cooresponding to each of the feature layers, a dataframe is to be created
df = pd.DataFrame(columns = ["feat_ID", "volume (in cubic m)"])

for i in range(len(gdf)):
    gdf_sub = gpd.GeoDataFrame(index = [i], 
                           crs = crs_sub, 
                           geometry = [gdf.iloc[i].geometry])
    dem_clip = dem.rio.clip(gdf_sub.geometry)
    # If we need to save the clipped file . To save the rioxrray file
    # dem_clip.rio.to_raster("dem01_projected_clip01.tif")
    # Calculation of the surface volume below certain height
    data = dem_clip.data[0].astype('float')
    data[(data < -9000) | (data > 9000)] = np.nan # -/+9000 m some random number. Mostly to filter out the void values
    base_height = np.nanpercentile(data, 50) # Here I have chosen the median value
    pixel_area = dem_clip.rio.resolution()[0] ** 2
    surface_volume = np.nansum((data < base_height) * (base_height - data) * pixel_area) # in cubic meter
    # store the result
    df.loc[i, "feat_ID"] = gdf.loc[i, "id"]
    df.loc[i, "volume (in cubic m)"] = surface_volume

In [ ]:
# To save the dataframe with result as xlsx file
df.to_csv("surface_volume.csv", index = False)

In [1]:
%load_ext watermark

%watermark -v -m -p numpy,matplotlib,geopandas,pandas,rioxarray,watermark

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

numpy     : 1.26.4
matplotlib: 3.8.4
geopandas : 0.14.4
pandas    : 2.2.2
rioxarray : 0.15.5
watermark : 2.4.3

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 32
Architecture: 64bit

